In [1]:
from u import *
from ut import *
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from distiller.quantization import PostTrainLinearQuantizer, LinearQuantMode
from copy import deepcopy

%load_ext autoreload
%autoreload 2

decoder = (Cache / 'vocab.npy').load()
encoder = get_encoder(decoder)
n_vocab = len(decoder)

In [18]:
c_base = Config(Wiki / 'shallow,layers_8,cache', device='cuda', logger=False).load()
import copy
c = copy.deepcopy(c_base).var(res=Wiki / 'quant_aware').unvar(
    'adaptive_ratio', 'pos_emb', 'hebbian', 'fix_softmax', 'hebbian_T', 'hebbian_gamma'
)
c = c.var(
    model=Proj / 'quantized_model.py',
    compress=c.res / 'distiller.yaml',
    train_batch=10,
    steps_per_epoch=1,
    step_warmup=0,
    lr=0.0001
).save(True)
# state = c_base.load_state('max')
# del state['amp']
# del state['opt']
# state['step'] = 0
# # state['opt']['param_groups'][0]['lr'] = 5e-4
# c.save_state(state['step'], state)

In [30]:
from quantized_model import get_net
net = get_net(c)
net, step = c.init_model(net, step='max', train=False)
print('Loaded model at step', step)

Loaded model at step 0


In [31]:
from data import *
from main import evaluate
data_val = SequentialIterator(c, c.eval_batch, split='valid')

In [3]:
print(c.train(env_gpu=0, steps=300000))

cd /data/scratch/zxyan/micronet/wikitext-103/quant_aware
CUDA_VISIBLE_DEVICES=0 python3 ../../quantized_model.py . steps=300000 opt_level=O0


In [32]:
evaluate(c, data_val, net)

{'loss': 3.466387987136841, 'perplexity': 32.02087350369042, 'time': 2.0}

In [33]:
print(c.train(env_gpu=1, steps=200000))

'/data/scratch/zxyan/micronet/wikitext-103/quant_aware'

In [4]:
quantization = (Wiki / 'quantization').mk()

In [5]:
import os
import distiller
from distiller.data_loggers import QuantCalibrationStatsCollector, collector_context

distiller.utils.assign_layer_fq_names(net)
collector = QuantCalibrationStatsCollector(net)

pretr_stats = quantization / 'pretrained_stats.yaml'
if not pretr_stats.exists():
    with collector_context(collector) as collector:
        evaluate(c, data_val, net)
        collector.save(pretr_stats)

In [6]:
quantizer = PostTrainLinearQuantizer(
    deepcopy(net),
    model_activation_stats=pretr_stats
)

In [7]:
stats_before_prepare = deepcopy(quantizer.model_activation_stats)
dummy_input = (torch.zeros(1, 1).to(dtype=torch.long),) * 2
quantizer.prepare_model(dummy_input)

RuntimeError: Unsupported prim::Constant kind: `s`. Send a bug report.

In [10]:
next(net.parameters()).dtp

torch.float32